# Подключения и функции

In [26]:
# Импорт

import pandas as pd
from clickhouse_driver import Client
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Функция создания датафрейма

def create_dataframe(user, password, table_name='films'):
    client = Client('90.156.216.60', user=user, password=password, database='it_minimalist_CH')
    query = f'SELECT * FROM {table_name}'
    df = client.query_dataframe(query)
    return df

В ячейке ниже создается датафрейм в переменной `df`.
<br>

- Замените `'your_username'`, `'your_password'` на ваши учетные данные, которые получили на почту (у каждого своя учетная запись с индивидуальным доступом к БД Clickhouse).
<br>
- Как вы заметили в параметрах функции `create_dataframe`, по умолчанию датафрейм создатеся из таблицы `'films'`. Также в базе данных лежит таблица `'payments'`

In [6]:
payments = create_dataframe(user='tanya25pm', password='klcemshu', table_name='payments')
films = create_dataframe(user='tanya25pm', password='klcemshu', table_name='films')

# Семинар 3

## Время

In [22]:
payments.pay_date = pd.to_datetime(payments.pay_date)

In [20]:
payments.pay_date = payments.pay_date.dt.date


In [23]:
payments.pay_date.dt.round('h')

0      2024-04-24
1      2024-03-03
2      2024-02-09
3      2024-04-23
4      2024-01-29
          ...    
3957   2024-02-11
3958   2024-04-08
3959   2024-01-16
3960   2024-04-15
3961   2024-04-05
Name: pay_date, Length: 3962, dtype: datetime64[ns]

In [10]:
payments.head(2)

,pay_date,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,2,дневной,NL42,1.0
1,2024-03-03,15,вечерний,HW75,3.0


In [ ]:
payments['deadline'] = '14-06-2024'

In [49]:
payments = payments.assign(deadline='14-06-2024')

In [51]:
payments.deadline = pd.to_datetime(payments.deadline)

C:\Users\User\AppData\Local\Temp\ipykernel_12760\320405444.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  payments.deadline = pd.to_datetime(payments.deadline)


In [54]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   pay_date       3938 non-null   datetime64[ns]
 1   film_id        3962 non-null   int64         
 2   part_of_day    3962 non-null   object        
 3   client_id      3962 non-null   object        
 4   count_tickets  3943 non-null   float64       
 5   deadline       3962 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 185.8+ KB


In [55]:
payments['result_day'] = payments.deadline - payments.pay_date

In [57]:
payments.head(2)

,pay_date,film_id,part_of_day,client_id,count_tickets,deadline,result_day
0,2024-04-24,2,дневной,NL42,1.0,2024-06-14,51 days
1,2024-03-03,15,вечерний,HW75,3.0,2024-06-14,103 days


In [59]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   pay_date       3938 non-null   datetime64[ns] 
 1   film_id        3962 non-null   int64          
 2   part_of_day    3962 non-null   object         
 3   client_id      3962 non-null   object         
 4   count_tickets  3943 non-null   float64        
 5   deadline       3962 non-null   datetime64[ns] 
 6   result_day     3938 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2), timedelta64[ns](1)
memory usage: 216.8+ KB


In [64]:
payments.result_day.dt.total_seconds()

0        4406400.0
1        8899200.0
2       10886400.0
3        4492800.0
4       11836800.0
           ...    
3957    10713600.0
3958     5788800.0
3959    12960000.0
3960     5184000.0
3961     6048000.0
Name: result_day, Length: 3962, dtype: float64

In [70]:
pd.to_datetime(payments.pay_date, utc=True)

0      2024-04-24 00:00:00+00:00
1      2024-03-03 00:00:00+00:00
2      2024-02-09 00:00:00+00:00
3      2024-04-23 00:00:00+00:00
4      2024-01-29 00:00:00+00:00
                  ...           
3957   2024-02-11 00:00:00+00:00
3958   2024-04-08 00:00:00+00:00
3959   2024-01-16 00:00:00+00:00
3960   2024-04-15 00:00:00+00:00
3961   2024-04-05 00:00:00+00:00
Name: pay_date, Length: 3962, dtype: datetime64[ns, UTC]

## Query

In [76]:
payments.query('count_tickets >= 5 or part_of_day == "вечерний"')

,pay_date,film_id,part_of_day,client_id,count_tickets,deadline,result_day
1,2024-03-03,15,вечерний,HW75,3.0,2024-06-14,103 days
2,2024-02-09,43,вечерний,XX54,5.0,2024-06-14,126 days
3,2024-04-23,19,вечерний,QR90,3.0,2024-06-14,52 days
4,2024-01-29,34,вечерний,SG93,5.0,2024-06-14,137 days
7,2024-03-24,13,дневной,ZJ39,5.0,2024-06-14,82 days
...,...,...,...,...,...,...,...
3949,2024-04-16,18,дневной,VB31,5.0,2024-06-14,59 days
3950,2024-01-08,6,вечерний,OO94,4.0,2024-06-14,158 days
3956,2024-02-29,7,вечерний,KC99,2.0,2024-06-14,106 days
3960,2024-04-15,28,вечерний,TU86,5.0,2024-06-14,60 days


## .merge (или join)
###### (джоин или слияние таблиц)

In [91]:
tabel_1 = payments[payments['client_id']=='YN41'].tail(6)
tabel_1 = tabel_1.bfill()
tabel_1

,pay_date,film_id,part_of_day,client_id,count_tickets
2653,2024-02-14,36,утренний,YN41,5.0
2683,2024-02-19,9,вечерний,YN41,2.0
2690,2024-04-28,42,дневной,YN41,3.0
3068,2024-04-28,10,дневной,YN41,2.0
3207,2024-03-13,6,утренний,YN41,2.0
3843,2024-05-01,16,дневной,YN41,4.0


In [90]:
tabel_2 = films[films['film_id'].isin([1, 2, 3, 4, 16, 42])]
tabel_2

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма"
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма"
15,16,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика"
41,42,Одержимость,290,2014,Дамьен Шазель,"драма, музыкальный фильм"
53,16,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика"


### 1. inner
<div>
<img src="attachment:72f66736-feb9-448f-9c9c-48ec111a2548.png" width="480"/>
</div>

In [92]:
inner = tabel_1.merge(tabel_2, on='film_id', how='inner')
inner

,pay_date,film_id,part_of_day,client_id,count_tickets,film_name,price_ticket,release_year,filmmaker,kind_film
0,2024-04-28,42,дневной,YN41,3.0,Одержимость,290,2014,Дамьен Шазель,"драма, музыкальный фильм"
1,2024-05-01,16,дневной,YN41,4.0,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика"
2,2024-05-01,16,дневной,YN41,4.0,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика"


### 2. left 
<div>
<img src="attachment:6eb451ad-e025-4a97-bb86-0643b6bf9eb9.png" width="480"/>
</div>

In [95]:
left = tabel_1.merge(tabel_2, on='film_id', how='left', indicator=True)
left.style.highlight_null(color='orange')

,pay_date,film_id,part_of_day,client_id,count_tickets,film_name,price_ticket,release_year,filmmaker,kind_film,_merge
0,2024-02-14,36,утренний,YN41,5.000000,nan,nan,nan,nan,nan,left_only
1,2024-02-19,9,вечерний,YN41,2.000000,nan,nan,nan,nan,nan,left_only
2,2024-04-28,42,дневной,YN41,3.000000,Одержимость,290.000000,2014.000000,Дамьен Шазель,"драма, музыкальный фильм",both
3,2024-04-28,10,дневной,YN41,2.000000,nan,nan,nan,nan,nan,left_only
4,2024-03-13,6,утренний,YN41,2.000000,nan,nan,nan,nan,nan,left_only
5,2024-05-01,16,дневной,YN41,4.000000,Матрица,290.000000,1999.000000,Энди и Ларри Вачовски,"боевик, научная фантастика",both
6,2024-05-01,16,дневной,YN41,4.000000,Матрица,290.000000,1999.000000,Энди и Ларри Вачовски,"боевик, научная фантастика",both


### 3. right 
<div>
<img src="attachment:ad6d98d1-2d5b-42b1-ac2b-381023512de4.png" width="480"/>
</div>

In [96]:
right = tabel_1.merge(tabel_2, on='film_id', how='right', indicator=True)
right.style.highlight_null(color='orange')

,pay_date,film_id,part_of_day,client_id,count_tickets,film_name,price_ticket,release_year,filmmaker,kind_film,_merge
0,nan,1,nan,nan,nan,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,right_only
1,nan,2,nan,nan,nan,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",right_only
2,nan,3,nan,nan,nan,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",right_only
3,nan,4,nan,nan,nan,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",right_only
4,2024-05-01,16,дневной,YN41,4.000000,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика",both
5,2024-04-28,42,дневной,YN41,3.000000,Одержимость,290,2014,Дамьен Шазель,"драма, музыкальный фильм",both
6,2024-05-01,16,дневной,YN41,4.000000,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика",both


### 4. outer 
<div>
<img src="attachment:a04afcc5-9795-48c9-9765-b336f400f858.png" width="480"/>
</div>

In [98]:
outer = tabel_1.merge(tabel_2, on='film_id', how='outer', indicator=True)
outer.style.highlight_null(color='orange')

,pay_date,film_id,part_of_day,client_id,count_tickets,film_name,price_ticket,release_year,filmmaker,kind_film,_merge
0,nan,1,nan,nan,nan,Побег из Шоушенка,190.000000,1994.000000,Фрэнк Дарабонт,драма,right_only
1,nan,2,nan,nan,nan,Крёстный отец,190.000000,1972.000000,Фрэнсис Форд Коппола,"детектив, драма",right_only
2,nan,3,nan,nan,nan,Тёмный рыцарь,340.000000,2008.000000,Кристофер Нолан,"боевик, детектив, драма",right_only
3,nan,4,nan,nan,nan,Крёстный отец 2,290.000000,1974.000000,Фрэнсис Форд Коппола,"детектив, драма",right_only
4,2024-03-13,6,утренний,YN41,2.000000,nan,nan,nan,nan,nan,left_only
5,2024-02-19,9,вечерний,YN41,2.000000,nan,nan,nan,nan,nan,left_only
6,2024-04-28,10,дневной,YN41,2.000000,nan,nan,nan,nan,nan,left_only
7,2024-05-01,16,дневной,YN41,4.000000,Матрица,290.000000,1999.000000,Энди и Ларри Вачовски,"боевик, научная фантастика",both
8,2024-05-01,16,дневной,YN41,4.000000,Матрица,290.000000,1999.000000,Энди и Ларри Вачовски,"боевик, научная фантастика",both
9,2024-02-14,36,утренний,YN41,5.000000,nan,nan,nan,nan,nan,left_only


### 5. cross
<div>
<img src="attachment:00823bd1-4868-4499-8f4a-3e4873f16b81.png" width="256"/>
</div>
<div>
<img src="attachment:f68e3391-005d-435b-b37c-5a08769b36c9.png" width="96"/>
</div>

In [99]:
cross = tabel_1.merge(tabel_2, how='cross')
cross

,pay_date,film_id_x,part_of_day,client_id,count_tickets,film_id_y,film_name,price_ticket,release_year,filmmaker,kind_film
0,2024-02-14,36,утренний,YN41,5.0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма
1,2024-02-14,36,утренний,YN41,5.0,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма"
2,2024-02-14,36,утренний,YN41,5.0,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"
3,2024-02-14,36,утренний,YN41,5.0,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма"
4,2024-02-14,36,утренний,YN41,5.0,16,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика"
5,2024-02-14,36,утренний,YN41,5.0,42,Одержимость,290,2014,Дамьен Шазель,"драма, музыкальный фильм"
6,2024-02-14,36,утренний,YN41,5.0,16,Матрица,290,1999,Энди и Ларри Вачовски,"боевик, научная фантастика"
7,2024-02-19,9,вечерний,YN41,2.0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма
8,2024-02-19,9,вечерний,YN41,2.0,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма"
9,2024-02-19,9,вечерний,YN41,2.0,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"


In [101]:
print(f'Количество строк в tabel_1: {tabel_1.shape[0]}')
print(f'Количество строк в tabel_2: {tabel_2.shape[0]}')

print(f'Количество строк при cross join: {tabel_1.shape[0] * tabel_2.shape[0]}')

Количество строк в tabel_1: 6
Количество строк в tabel_2: 7
Количество строк при cross join: 42


## .pivot и .T

### .pivot

In [104]:
payments.head(3)

,pay_date,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,2,дневной,NL42,1.0
1,2024-03-03,15,вечерний,HW75,3.0
2,2024-02-09,43,вечерний,XX54,5.0


In [103]:
payments.pivot_table(index='client_id', columns='part_of_day', values='count_tickets', aggfunc='sum')

part_of_day,вечерний,дневной,утренний
client_id,,,
AD64,30.0,24.0,15.0
AD97,13.0,27.0,42.0
AF29,20.0,17.0,16.0
AI63,21.0,23.0,33.0
AL42,25.0,11.0,15.0
...,...,...,...
ZQ81,19.0,25.0,17.0
ZR11,17.0,29.0,26.0
ZS96,24.0,11.0,10.0


In [108]:
agg = payments.groupby(['client_id','part_of_day'])['count_tickets'].sum().reset_index()
agg

,client_id,part_of_day,count_tickets
0,AD64,вечерний,30.0
1,AD64,дневной,24.0
2,AD64,утренний,15.0
3,AD97,вечерний,13.0
4,AD97,дневной,27.0
...,...,...,...
592,ZU59,дневной,20.0
593,ZU59,утренний,24.0
594,ZV13,вечерний,21.0
595,ZV13,дневной,10.0


In [110]:
agg.pivot(index='client_id', columns='part_of_day', values='count_tickets')

part_of_day,вечерний,дневной,утренний
client_id,,,
AD64,30.0,24.0,15.0
AD97,13.0,27.0,42.0
AF29,20.0,17.0,16.0
AI63,21.0,23.0,33.0
AL42,25.0,11.0,15.0
...,...,...,...
ZQ81,19.0,25.0,17.0
ZR11,17.0,29.0,26.0
ZS96,24.0,11.0,10.0


### .T

In [117]:
t_show = agg.head(5)
t_show

,client_id,part_of_day,count_tickets
0,AD64,вечерний,30.0
1,AD64,дневной,24.0
2,AD64,утренний,15.0
3,AD97,вечерний,13.0
4,AD97,дневной,27.0


In [116]:
t_show.T

,0,1,2,3,4
client_id,AD64,AD64,AD64,AD97,AD97
part_of_day,вечерний,дневной,утренний,вечерний,дневной
count_tickets,30.0,24.0,15.0,13.0,27.0


# Домашнее задание по семинару 👨‍💻

#### 1. Поработаем с дата-время в csv файле `time.csv`
1. Прочитайте файл `time.csv`
2. Преобразуйте столбцы `start_time` и `finish_time` в формат datetime
3. Создайте новый столбец, в котором будет разница `finish_time` и `start_time`
4. Создайте новый столбец, в котором будет кол-во секунд между `finish_time` и `start_time`
5. Создайте столбец `round_finish_time`, в котором будет округленное время до часа из столбца `finish_time`

In [24]:
# time.csv файлик для этого ДЗ
df_time = pd.read_csv('time.csv')
df_time.head(5)

,start_time,finish_time
0,2024-05-02 18:22:11,2024-05-02 18:27:27
1,2024-05-10 13:43:07,2024-05-10 13:51:50
2,2024-05-05 08:51:18,2024-05-05 08:55:03
3,2024-05-15 16:30:07,2024-05-15 16:49:35
4,2024-06-15 17:29:44,2024-08-31 20:59:59


In [29]:
# your code переводим столбцы  в формат datetime64[ns]
df_time['start_time'] = pd.to_datetime(df_time['start_time'])
df_time['finish_time'] = pd.to_datetime(df_time['finish_time'])
df_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   start_time   15 non-null     datetime64[ns]
 1   finish_time  15 non-null     datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 368.0 bytes


In [33]:
# находим разницу между столбцами
df_time['delta_time'] = df_time['finish_time'] - df_time['start_time']

In [36]:
# добавим столбец с разницей в секундах
df_time['delta_time_seconds'] = df_time['delta_time'].dt.total_seconds()


In [40]:
# добавляем столбец с округлением до часа финального времени
df_time['round_finish_time'] = df_time['finish_time'].dt.round('h')
df_time

,start_time,finish_time,delta_time,delta_time_seconds,round_finish_time
0,2024-05-02 18:22:11,2024-05-02 18:27:27,0 days 00:05:16,316.0,2024-05-02 18:00:00
1,2024-05-10 13:43:07,2024-05-10 13:51:50,0 days 00:08:43,523.0,2024-05-10 14:00:00
2,2024-05-05 08:51:18,2024-05-05 08:55:03,0 days 00:03:45,225.0,2024-05-05 09:00:00
3,2024-05-15 16:30:07,2024-05-15 16:49:35,0 days 00:19:28,1168.0,2024-05-15 17:00:00
4,2024-06-15 17:29:44,2024-08-31 20:59:59,77 days 03:30:15,6665415.0,2024-08-31 21:00:00
5,2024-05-02 04:20:40,2024-05-02 04:33:54,0 days 00:13:14,794.0,2024-05-02 05:00:00
6,2024-06-02 07:37:28,2024-06-02 08:37:28,0 days 01:00:00,3600.0,2024-06-02 09:00:00
7,2024-05-03 15:10:55,2024-05-03 15:16:59,0 days 00:06:04,364.0,2024-05-03 15:00:00
8,2024-06-03 12:48:46,2024-06-03 12:56:49,0 days 00:08:03,483.0,2024-06-03 13:00:00
9,2024-05-01 13:04:13,2024-05-01 13:23:33,0 days 00:19:20,1160.0,2024-05-01 13:00:00


#### 2. Теперь поджоиним (merge)
1. Инициализиируйте два датафрейма `df_A` и `df_B` в следующей ячейке
2. Присоедините к `df_A` датафрейм `df_B`, используя left join. **По одному** ключу key1
3. Присоедините к `df_A` датафрейм `df_B`, используя left join. **По двум** ключам key1 и key2
4. Напишите в чем разница 

In [42]:
df_A = pd.DataFrame(
   {
      "key1": ["Даня", "Даня", "Артем", "Катя"],
      "key2": ["K0", "K1", "K0", "K1"],
      "A": ["A0", "A1", "A2", "A3"],
      "B": ["B0", "B1", "B2", "B3"],
   }
)

df_B = pd.DataFrame(
   {
      "key1": ["Даня", "Андрей", "Артем", "Катя"],
      "key2": ["K0", "K0", "K0", "K0"],
      "C": ["C0", "C1", "C2", "C3"],
      "D": ["D0", "D1", "D2", "D3"],
   }
)

In [45]:
# your code 
# поскольку по key1 у нас совпали все данные и у обоих таблиц имеется одинковый столбец key2,
# вторая таблица присоединилась полностью, а наши столбцы key2 получили индексы, чтобы мы их могли различить между собой

df_key_1 = df_A.merge(df_B, how= 'left', on= 'key1')
df_key_1

,key1,key2_x,A,B,key2_y,C,D
0,Даня,K0,A0,B0,K0,C0,D0
1,Даня,K1,A1,B1,K0,C0,D0
2,Артем,K0,A2,B2,K0,C2,D2
3,Катя,K1,A3,B3,K0,C3,D3


In [48]:
# при объединении сразу по двум ключам получили NaN, т.к. по key2 наши данные не совпали
df_keys = df_A.merge(df_B, how= 'left', on= ['key1', 'key2'])
df_keys


,key1,key2,A,B,C,D
0,Даня,K0,A0,B0,C0,D0
1,Даня,K1,A1,B1,NaN,NaN
2,Артем,K0,A2,B2,C2,D2
3,Катя,K1,A3,B3,NaN,NaN
